In [ ]:
import pandas as pd
import os
import glob

# Set your folder INPUT path here
folder_paths = ["cloudfront-top-referrers-history","cloudfront-popular-objects-history"]

output_folder_path = "cloudfront-historical-data"

for folder_path in folder_paths: 
    files = glob.glob(os.path.join(folder_path, "*.csv")) + glob.glob(os.path.join(folder_path, "*.xlsx"))
    print(f"Found {len(files)} files to process")
    
    for file_path in files:
        print(f"\nProcessing: {os.path.basename(file_path)}")
        
        try:
            # Read first few rows to get metadata (rows 2-5, which are indices 1-4)
            if file_path.endswith('.xlsx'):
                meta_df = pd.read_excel(file_path, header=None, nrows=6)
            else:
                meta_df = pd.read_csv(file_path, header=None, nrows=6)
            
            # Extract metadata from rows 2-5
            metadata = {}
            for i in range(1, 5):  # rows 2-5 (indices 1-4)
                if i < len(meta_df):
                    key = str(meta_df.iloc[i, 0]) if pd.notna(meta_df.iloc[i, 0]) else f"field_{i}"
                    value = str(meta_df.iloc[i, 1]) if len(meta_df.columns) > 1 and pd.notna(meta_df.iloc[i, 1]) else ""
                    metadata[key] = value
            
            print(f"Metadata: {metadata}")
            
            # Read the actual data starting from row 7 (skiprows=6)
            if file_path.endswith('.xlsx'):
                df = pd.read_excel(file_path, skiprows=7)
            else:
                df = pd.read_csv(file_path, skiprows=7)
            
            # Clean column names
            df.columns = df.columns.str.strip().str.replace('"', '')
            
            # Add metadata columns
            for key, value in metadata.items():
                df[key] = value
            
            # Save as Parquet
            output_filename = os.path.splitext(os.path.basename(file_path))[0] + ".parquet"
            output_path = os.path.join(output_folder_path, folder_path)
            os.makedirs(output_path, exist_ok=True)
            output_path = os.path.join(output_path, output_filename)
            
            df.to_parquet(output_path, index=False)
            
            print(f"Saved {len(df)} rows to {output_filename}")
            
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

print("\nDone!")

Found 19 files to process

Processing: TopReferrers-2025-06-17-14-08-47.csv
Metadata: {'Report': 'TopReferrers', 'DistributionID': 'ALL', 'StartDateUTC': '2025-05-30T00:00:00.000Z', 'EndDateUTC': '2025-06-05T23:59:59.000Z'}
Saved 25 rows to TopReferrers-2025-06-17-14-08-47.parquet

Processing: TopReferrers-2025-07-17-14-36-50.csv
Metadata: {'Report': 'TopReferrers', 'DistributionID': 'ALL', 'StartDateUTC': '2025-07-10T14:36:46.779Z', 'EndDateUTC': '2025-07-17T14:36:46.779Z'}
Saved 25 rows to TopReferrers-2025-07-17-14-36-50.parquet

Processing: TopReferrers-2025-07-10-14-25-28.csv
Metadata: {'Report': 'TopReferrers', 'DistributionID': 'ALL', 'StartDateUTC': '2025-07-03T14:25:25.433Z', 'EndDateUTC': '2025-07-10T14:25:25.433Z'}
Saved 25 rows to TopReferrers-2025-07-10-14-25-28.parquet

Processing: TopReferrers-2025-06-17-14-09-03.csv
Metadata: {'Report': 'TopReferrers', 'DistributionID': 'ALL', 'StartDateUTC': '2025-06-06T00:00:00.000Z', 'EndDateUTC': '2025-06-12T23:59:59.000Z'}
Saved 25

## Testing

In [36]:
for folder_path in folder_paths: 
    parquet_folder = f"{output_folder_path}/{folder_path}"
    parquet_files = glob.glob(os.path.join(parquet_folder, "*.parquet"))
    
    print(f"Found {len(parquet_files)} parquet files")
    
    # Read and combine all parquet files
    all_dfs = []
    for file_path in parquet_files:
        df = pd.read_parquet(file_path)
        print(f"Loaded {os.path.basename(file_path)}: {len(df)} rows, {len(df.columns)} columns")
        all_dfs.append(df)
    
    # Combine into single dataframe
    combined_df = pd.concat(all_dfs, ignore_index=True)
    display(combined_df)

Found 17 parquet files
Loaded TopReferrers-2025-04-25-15-21-12.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-08-14-17-08-07.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-07-31-17-50-16.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-08-07-15-07-43.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-06-17-14-08-47.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-07-17-14-36-50.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-07-24-14-59-21.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-05-29-15-33-51.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-07-10-14-25-28.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-04-22-19-46-31.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-07-03-14-05-53.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-05-15-17-10-30.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-05-22-20-13-16.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-06-20-15-23-05.parquet: 25 rows, 8 columns
Loaded TopReferrers-2025-

,DistributionID,FriendlyName,Referrers,RequestCount,RequestsError,Report,StartDateUTC,EndDateUTC
0,ALL,ALL,www.canada.ca,30522,0,TopReferrers,2025-04-18T15:21:05.831Z,2025-04-25T15:21:05.831Z
1,ALL,ALL,cdn.design-system.alpha.canada.ca,29038,0,TopReferrers,2025-04-18T15:21:05.831Z,2025-04-25T15:21:05.831Z
2,ALL,ALL,-,5787,0,TopReferrers,2025-04-18T15:21:05.831Z,2025-04-25T15:21:05.831Z
3,ALL,ALL,digital.canada.ca,5453,0,TopReferrers,2025-04-18T15:21:05.831Z,2025-04-25T15:21:05.831Z
4,ALL,ALL,localhost,4964,0,TopReferrers,2025-04-18T15:21:05.831Z,2025-04-25T15:21:05.831Z
...,...,...,...,...,...,...,...,...
420,ALL,ALL,author-canada-prod.adobecqms.net,269,0,TopReferrers,2025-06-06T00:00:00.000Z,2025-06-12T23:59:59.000Z
421,ALL,ALL,dev-myevents.bac-lac.gc.ca,267,0,TopReferrers,2025-06-06T00:00:00.000Z,2025-06-12T23:59:59.000Z
422,ALL,ALL,dv25.openplus.ca,243,0,TopReferrers,2025-06-06T00:00:00.000Z,2025-06-12T23:59:59.000Z
423,ALL,ALL,dv26.openplus.ca,199,0,TopReferrers,2025-06-06T00:00:00.000Z,2025-06-12T23:59:59.000Z


Found 22 parquet files
Loaded PopularObjects-2025-04-25-15-20-28.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-03-07-18-18-34.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-06-17-14-13-19.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-07-03-14-05-47.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-04-25-15-23-35.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-05-01-14-48-27.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-06-17-14-13-35.parquet: 50 rows, 17 columns
Loaded PopularObjects-2024-10-31-15-01-42.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-08-14-17-07-58.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-08-07-15-07-32.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-03-20-14-38-59.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-02-20-15-46-45.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-04-10-14-16-30.parquet: 50 rows, 17 columns
Loaded PopularObjects-2025-05-15-17-12-31.parquet: 50 

,DistributionID,FriendlyName,Object,RequestCount,HitCount,MissCount,HitCountPct,BytesFromMisses,TotalBytes,IncompleteDownloadCount,Http2xx,Http3xx,Http4xx,Http5xx,Report,StartDateUTC,EndDateUTC
0,E16K5WRSMCPC,E16K5WRSMCPC,/@cdssnc/gcds-components@latest/dist/gcds/gcds...,12100,11843,102,97.88Pct,398405,37546530,0,11140,959,1,0,PopularObjects,2025-04-18T15:19:06.152Z,2025-04-25T15:19:06.152Z
1,E16K5WRSMCPC,E16K5WRSMCPC,/@cdssnc/gcds-components@latest/dist/gcds/gcds...,11918,11687,101,98.06Pct,668927,73391113,3,11215,701,0,0,PopularObjects,2025-04-18T15:19:06.152Z,2025-04-25T15:19:06.152Z
2,E16K5WRSMCPC,E16K5WRSMCPC,/@cdssnc/gcds-components@latest/dist/gcds/p-4b...,11540,11310,98,98.01Pct,995764,107004931,5,10729,809,0,0,PopularObjects,2025-04-18T15:19:06.152Z,2025-04-25T15:19:06.152Z
3,E16K5WRSMCPC,E16K5WRSMCPC,/@cdssnc/gcds-components@latest/dist/gcds/p-e1...,11518,11300,89,98.11Pct,46228,5982981,3,10742,775,0,0,PopularObjects,2025-04-18T15:19:06.152Z,2025-04-25T15:19:06.152Z
4,E16K5WRSMCPC,E16K5WRSMCPC,/@cdssnc/gcds-components@latest/dist/gcds/p-6c...,8063,7841,91,97.25Pct,6677410,527886225,2,7588,471,0,0,PopularObjects,2025-04-18T15:19:06.152Z,2025-04-25T15:19:06.152Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,E16K5WRSMCPC,E16K5WRSMCPC,/@cdssnc/gcds-components@0.36.0/dist/gcds/p-e1...,343,334,5,97.38Pct,2745,159247,0,200,142,0,0,PopularObjects,2025-06-12T00:00:00.000Z,2025-06-19T00:00:00.000Z
1096,E16K5WRSMCPC,E16K5WRSMCPC,/@cdssnc/gcds-components@0.36.0/dist/gcds/p-75...,339,330,5,97.35Pct,40935,2028297,0,199,139,0,0,PopularObjects,2025-06-12T00:00:00.000Z,2025-06-19T00:00:00.000Z
1097,E16K5WRSMCPC,E16K5WRSMCPC,/@cdssnc/gcds-components@0.32.0/dist/gcds/gcds...,339,289,13,85.25Pct,48062,962138,0,298,18,0,0,PopularObjects,2025-06-12T00:00:00.000Z,2025-06-19T00:00:00.000Z
1098,E16K5WRSMCPC,E16K5WRSMCPC,/@cdssnc/gcds-components@0.32.0/dist/gcds/p-4b...,333,282,12,84.68Pct,150448,2941927,0,293,17,0,0,PopularObjects,2025-06-12T00:00:00.000Z,2025-06-19T00:00:00.000Z
